In [54]:
import cv2
import numpy as np
thresh = 128
max_thresh = 255  

In [55]:
def threshold_callback1(x, img):
    #Convert into binary image using thresholding
    #Documentation for threshold: http://docs.opencv.org/modules/imgproc/doc/miscellaneous_transformations.html?highlight=threshold#threshold
    #Example of thresholding: http://docs.opencv.org/doc/tutorials/imgproc/threshold/threshold.html
    _, thres_output = cv2.threshold(img, x, max_thresh, 0)
    cv2.namedWindow("Threshold", cv2.WINDOW_AUTOSIZE);
    cv2.imshow("Threshold", thres_output);

    #Documentation for adaptive thresholding: http://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html?highlight=adaptivethreshold#cv2.adaptiveThreshold
    #adaptiveThreshold(cbd.src_gray, a_thres_output, 255, ADAPTIVE_THRESH_MEAN_C, THRESH_BINARY, 17, -5);
    a_thres_output = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,17,-5)
    cv2.namedWindow("Adaptive threshold", cv2.WINDOW_AUTOSIZE);
    cv2.imshow("Adaptive threshold", a_thres_output);

    #Find contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours
    contours,heirarchy = cv2.findContours(a_thres_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);


    #find largest contour
    maxsize = 0;
    maxind = 0;
    for i in range(len(contours)):
    #Documentation on contourArea: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#
        area = cv2.contourArea(contours[i]);
        if (area > maxsize):
            maxsize = area
            maxind = i

    print("The area of the largest contour detected is: ", maxsize)
    print("-----------------------------")

    #Draw contours
    #Documentation for drawing contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=drawcontours#drawcontours
    #drawContours(contour_output, contours, maxind, Scalar(255, 0, 0), CV_FILLED, 8, hierarchy);
    contour_output = cv2.cvtColor(np.zeros(np.shape(thres_output), dtype='uint8'), cv2.COLOR_GRAY2BGR)    
    cv2.drawContours(contour_output, contours, -1, (0, 0, 255), 2, 8)

    #Show in a window
    cv2.namedWindow("Contours", cv2.WINDOW_AUTOSIZE);
    cv2.imshow("Contours", contour_output);

In [58]:
def threshold_callback2(x, img):
    #Parameters
    ksize = 3;
    scale = 1;
    delta = 0;
    ddepth = cv2.CV_8U;


    # check out "cv2.Sobel" for finding the gradients of x and y:
    # https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html?highlight=sobel#sobel
    #Gradient X
    gx = cv2.Sobel(img, ddepth, 1, 0)
    abs_grad_x = np.absolute(gx)
    abs_grad_x = np.uint8(255.*abs_grad_x/np.max(abs_grad_x))
    
    #Gradient Y
    gy = cv2.Sobel(img, ddepth, 0, 1)
    abs_grad_y = np.absolute(gy)
    abs_grad_y = np.uint8(255.*abs_grad_y/np.max(abs_grad_y))

    #Total Gradient (Approximately)
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

    cv2.namedWindow("Edge", cv2.WINDOW_AUTOSIZE)
    cv2.imshow("Edge", grad);

    #Thresholding
    _, thres = cv2.threshold(grad, x, max_thresh, 0)
    cv2.namedWindow("Edge threshold", cv2.WINDOW_AUTOSIZE)
    cv2.imshow("Edge threshold", thres)


    #Find contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours
    contours,heirarchy = cv2.findContours(thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    
    print("The number of contours detected is: ",len(contours))


    #Find largest contour
    maxsize = 0;
    maxind = 0;
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > maxsize): 
            maxsize = area;
            maxind = i;

    print("The area of the largest contour detected is: ", cv2.contourArea(contours[maxind]))
    print("-----------------------------")

    #Draw contours
    #Documentation for drawing contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=drawcontours#drawcontours
    contour_output = cv2.cvtColor(np.zeros(np.shape(thres), dtype='uint8'), cv2.COLOR_GRAY2BGR)    
    cv2.drawContours(contour_output, contours, -1, (0, 0, 255), 2, 8)
    #Show in a window
    cv2.namedWindow("Contours2", cv2.WINDOW_AUTOSIZE);
    cv2.imshow("Contours2", contour_output);


In [59]:

#Load source image and convert it to gray
src = cv2.imread("bat.png", 1)

#Create Window and display source image
cv2.namedWindow("Source", cv2.WINDOW_AUTOSIZE)
cv2.imshow("Source", src);

#Convert image to gray
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
#Blur the image
blur = cv2.blur(gray, (5, 5))

#Create Trackbar
#Documentation for createTrackbar: http://docs.opencv.org/modules/highgui/doc/user_interface.html?highlight=createtrackbar#createtrackbar
#Example of adding a trackbar: http://docs.opencv.org/doc/tutorials/highgui/trackbar/trackbar.html

#Method 1: Adaptive thresholding
#cv2.createTrackbar("Threshold:", "Source", thresh, max_thresh, lambda x: threshold_callback1(x, blur))
#createTrackbar("Threshold1:", "Source", &cbd.thresh, cbd.max_thresh, threshold_callback1, &cbd);
#threshold_callback1(0, &cbd);

#Method 2: Edge thresholding
cv2.createTrackbar("Threshold:", "Source", thresh, max_thresh, lambda x: threshold_callback2(x, blur))
#createTrackbar("Threshold2:", "Source", &cbd.thresh, cbd.max_thresh, threshold_callback2, &cbd);
#threshold_callback2(0, &cbd);

#Wait until keypress
cv2.waitKey(0);
cv2.destroyAllWindows()

The number of contours detected is:  12
The area of the largest contour detected is:  71.5
-----------------------------
